In [5]:
from diffusion import experiments
from diffusion.experiments import InferenceExperiment, InferenceExperimentConfiguration
import logging
import os
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("mps")
prompt = "a photo of an astronaut riding a horse on mars"

# set notebook autoload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:


scheduler = EulerDiscreteScheduler.from_config("stabilityai/stable-diffusion-2", subfolder= "scheduler")
pipe = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2")



Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00, 20.94it/s]


In [8]:
pipe.to(device)
image = pipe(prompt, height=512, width=512).images[0]

display(image)

 14%|█▍        | 7/50 [00:05<00:35,  1.21it/s]


KeyboardInterrupt: 

In [6]:
pipe = StableDiffusionPipeline.from_pretrained('stable-diffusion-v1-5/stable-diffusion-v1-5')
pipe.to(device)
image = pipe(prompt).images[0]
display(image)

 30%|███       | 15/50 [00:17<00:41,  1.17s/it]


KeyboardInterrupt: 

In [2]:


config = InferenceExperimentConfiguration()

curr_path = "./"

config.path = os.path.join(curr_path, "inference_data")
config.name = "inference_experiment"
config.overwrite_results = True
experiment = InferenceExperiment(logger, config)
experiment.run()

Generating experiments with factors: ['image_size', 'num_inf_steps'] and levels: [[(512, 512)], [25]]
INFO:__main__:Generating experiments with factors: ['image_size', 'num_inf_steps'] and levels: [[(512, 512)], [25]]
DoE: [[0. 0.]]
INFO:__main__:DoE: [[0. 0.]]
Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  4.24it/s]
Running experiments

INFO:__main__:Running experiments

Running experiment: {'image_size': (512, 512), 'num_inf_steps': 25}
INFO:__main__:Running experiment: {'image_size': (512, 512), 'num_inf_steps': 25}
  0%|          | 0/1 [00:00<?, ?it/s]Running batch with prompt: ['a kitchen', 'space']
INFO:__main__:Running batch with prompt: ['a kitchen', 'space']
100%|██████████| 1/1 [00:39<00:00, 39.40s/it]
Saving results
INFO:__main__:Saving results
Saving results to ./inference_data/repetition_0__image_size_(512, 512)__num_inf_steps_25/results.json
INFO:__main__:Saving results to ./inference_data/repetition_0__image_size_(512, 512)__num_inf_steps_25/results.